# New Data Analysis

In [11]:
dataDir='Data/'

In [12]:
import json
import pandas as pd
import re

## Openning the data¶

In [13]:
#function that open the file and create a "good" dataset
def createDataFrame(file):
    ##lists of 50 data are loaded and then merged
    #create list of dataframes and open file
    dataframes=[]
    file=open(dataDir+file, 'r')
    
    #counter is important for the index
    c=0
    while True:
        #each line is 50 data (as limit=50 when downloading)
        txt=file.readline()
        #possibly stop
        if txt=='':
            break

        #load the data with right index
        j=json.loads(txt)
        l=len(j)
        dataframes.append(pd.DataFrame(j, index=list(range(c, c+l))))
        
        c+=l
    #merge all datasets and return result
    return pd.concat(dataframes)

In [14]:
dfEvent=createDataFrame('event.json')

In [15]:
dfEvent.columns

Index(['beneficiaries_id', 'canceled', 'checks_payable_to_address',
       'contributions_info', 'distribution_paid_for_by', 'end_date',
       'end_time', 'entertainment', 'hosts_id', 'id', 'is_presidential',
       'make_checks_payable_to', 'more_details', 'notes', 'other_members',
       'postponed', 'resource_uri', 'rsvp_info', 'start_date', 'start_time',
       'venue_id'],
      dtype='object')

In [16]:
#dfEvent

--
### Number of events:

In [17]:
#number of tracked events:
len(dfEvent)

24108

--
### Contributions for each event:

In [18]:
##quick look at what it looks like:
#dfEvent['contributions_info']

#### looking for host contribution:

see https://regex101.com/r/pj3UC4/1 for more info on the reg expr

In [32]:
def getHostContrib(s):
    reg=r'((h|H)ost[ Commitee\:]+\$?[,0-9]+|\$?[,0-9]+[ per\-PACto]+(h|H)ost( [^$]|;|\n|$)|(h|H)ost - \$[,0-9]+)'
    reg_expr=re.compile(reg)
    reg_expr2=re.compile(r"(h|H)ost")
    
    #determinating if there is a host contribution info:
    hosts = re.finditer(reg_expr2, s)
    h=False
    for hostNum, host in enumerate(hosts):
        h=True
        
    
    matches = re.finditer(reg_expr, s)
    c=0
    l=[]
    for matchNum, match in enumerate(matches):
        l.append(match.group())
        c+=1
    #c is the number of hosts info caught

    #if not found, but there is some host info
    if h:
        print(s)
        
        
        

#### Type of contribution_info string that gives me toubles:

In [33]:
dfEvent['contributions_info'].apply(getHostContrib)

Host: $2,500 - PAC | $1,000 - Personal / Attend: $1,000 - PAC | $500 Personal
Host: $2,500 - PAC | $1,500 - Individual / Attend: $1,000 - PAC | $500 Individual
Host: $2,500 - PAC | $1,500 - Personal / Attend: $1,500 - PAC | $1,000 Personal
Host: $2,500 - PAC | $1,500 - Personal / Attend: $1,500 - PAC | $1,000 Personal
Host: $2,500 / Attend: $1,000 / PAC or Personal: $500
PAC Host: $2,500 / PAC Guest: $1,000 / Individual Host: $1,000 / Individual Guest: $500
Host: $2,500 / Attend: $1,000 / PAC or Personal: $500
Host: $2,500 / PAC: $1,000 / Personal: $500
PAC Host: $2,500 / PAC Guest: $1,000 / Individual: $500
Host: $2,500 / Attend: $1,000 / PAC or Personal: $500
Host: $5,000 / Sponsor: $2,500 / PAC: $1,000 / Individual: $500
Host: $2,500 - PAC | $1,000 - Personal / Attend:  $1,000 - PAC | $500 - Personal
Host: $2,500 / Attend: $1,000 / PAC or Personal: $500
Host: $2,500 / Individual: $1,000 / Attend: $1,000 / PAC: $500
Co-Host: $5,000 PAC / $2,700 Individual / Guest: $1,000 PAC or Indiv

$5,000 Host | $2,500 Sponsor | $1,000 Guest 
$5,000 Individual Host | $2,500 Individual Sponsor | $1,000 Individual Guest 
Chair $5,000 Host $2,500 PAC $1,000
$100,000 Event Chair $33,400 Host Premium Seat $10,000 Preferred Seat $5,000 General Seat $2,700
$25,000 per person $100,000 per couple and spot on host committee
Chair $50,000 Host $33,400 Champion $10,000 Fighter $2,700
 PAC Host $2,500 ~ PAC Guest $1,000  Individual Host $1,000 ~ Individual Guest $500
PAC Host $2,000 PAC Guest $1,000 Individual Guest $500
Event Chairs $5400 Event Vice Chairs $2700 Platinum Hosts $1000 Gold Hosts $500 Silver Hosts $250 Guests $100 Per Person
$2,000 Sponsor $1,000 Host $500 Supporter $250 Friend
Host $10,000 Fighter $1,000 Champion $2,700
Sponsor $5,000 per PAC or $2,500 Individual Co-Host $2,500 PAC or $1,000 Individual Attend $1,000 per PAC or $500 Individual
Host $1,500 Patron $1,000 Sponsor $500 Friend $250
Event Chairs $5,400 Event Vice Chairs $2,700 Platinum Hosts $1,000 Gold Hosts $500 Si

$1,000 Host; $250 Attend
$2,500 Host; $1,000 PAC; $500 Person
$5,000 PAC Host; $2,500 PAC Sponsor; $1,000 PAC Supporter; $250 Personal Contribution
$2,500 Host; $1,000 Individual
$1,500 Host; $1,000 PAC; $500 Person
$2,500 Host; $1,000 PAC; $500 Personal
$2,500 Host; $1,000 PAC; $500 Individual; $250 YR
$2,500 Co-Host (4 tickets), $1,000 PAC; $250 Individual; $100 Young Republican; $75 Florida Young Republican
$2,500 PAC Host; $1,500 PAC Attendee; $500 Personal
$5,000 Host; $2,500 Host if Previously Contributed $2,500; $1,000 Attendee
$2,500 PAC Host; $1,000 PAC Guest; $1,000 Individual Host; $250 Individual Guest
$2,500 Host; $1,000 PAC; $500 Individual 
$2,500 Co-Host; $1,000 PAC; $500 Personal
$2,500 PAC Host; $1,000 PAC; $500 Personal
$2,500 PAC Host; $1,500 PAC Attendee; $500 Personal
$5,000 Host; $2,500 Sponsor; $1,000 Guest
$5,000 Host; $2,500 Sponsor; $1,000 PAC; $500 Individual
$2,500 PAC Host; $1,000 Individual Host; $1,000 PAC Attendee; $500 Individual Attendee
$5,000 Host; 

$2,000 PAC Co-Host; $1,000 Individual Co-Host; $1,000 PAC; $500 Individual
$2,000 PAC Co-Host; $1,000 Individual Co-Host; $1,000 PAC; $500 Individual
$2,000 PAC Co-Host; $1,000 Individual Co-Host; $1,000 PAC; $500 Individual
$5,000 Host; $2,500 PAC; $1,000 Individual
$5,000 Host; $2,500 Sponsor; $1,000 Guest
$5,000 Host; $2,500 Sponsor; $1,000 PAC; $500 Individual
$5,000 PAC Host; $2,400 Sponsor; $1,000 Amigo; $1,000 PAC Individual; $500 Supporter; $250 Individual
$1,000 PAC Host; $500 Individual; $250 Host; $100 Individual
$2,000 PAC Host; $1,000 Individual Host; $1,000 PAC; $500 Individual
$2,500 Co-Host; $1,000 PAC; $500 Individual
$5,000 Co-Host; $2,500 PAC; $1,000 Attendee
$10,000 Co-Host; $5,000 Supporter; $1,000 Donor
$5,000 PAC Host; $2,500 PAC Sponsor; $1,000 PAC Guest; $500 Individual
$2,500 PAC Host; $1,000 Personal Host; $1,000 PAC Sponsor; $500 Personal Sponsor; $100 Personal Murphy Enthusiast
$2,000 PAC Host; $1,000 Individual Host; $1,000 PAC; $500 Individual
$2,000 Host

TypeError: expected string or bytes-like object

In [34]:
f=open('pbs', 'r')
l=f.readlines()
f.close()
pbs=len(l)
del l
print('there are', pbs, 'problems over', len(dfEvent), 'cases, ie:', 100*pbs/len(dfEvent), '% of errors')

there are 4829 problems over 24108 cases, ie: 20.030695204911233 % of errors


I think that this is acceptable, isn't it?

# Open secret data

In [35]:
dfContrib=createDataFrame("candContrib.json")
dfContrib.columns

Index(['cid', 'indivs', 'org_name', 'pacs', 'total', 'year'], dtype='object')

In [109]:
dfContrib.total=dfContrib.total.apply(int)#.sort_index()#'total')

In [111]:
dfContrib.sort_values(by='total', ascending=False)

,cid,indivs,org_name,pacs,total,year
380,N00035267,154044,NorPAC,4827,158871,2014
400,N00035267,154044,NorPAC,4827,158871,2018
390,N00035267,154044,NorPAC,4827,158871,2016
401,N00035267,156910,"Paul, Weiss et al",0,156910,2018
391,N00035267,154150,"Paul, Weiss Et Al",0,154150,2016
381,N00035267,153900,"Paul, Weiss et al",0,153900,2014
402,N00035267,100950,Gibbons PC,10000,110950,2018
392,N00035267,100950,Gibbons PC,10000,110950,2016
350,N00000699,106657,Lowenstein Sandler LLP,0,106657,2014
360,N00000699,85550,NorPAC,5000,90550,2016


In [112]:
grouped=dfContrib['total'].groupby(dfContrib['year'])
grouped

In [115]:
grouped.describe(include='all')

year       
2014  count       130.000000
      mean      23400.784615
      std       27602.586754
      min        5000.000000
      25%       10000.000000
      50%       10999.000000
      75%       20845.250000
      max      158871.000000
2016  count       140.000000
      mean      24105.957143
      std       26270.845031
      min        8000.000000
      25%       10000.000000
      50%       13700.000000
      75%       20843.500000
      max      158871.000000
2018  count       140.000000
      mean      17938.892857
      std       26264.276344
      min        2500.000000
      25%        5400.000000
      50%        9200.000000
      75%       15000.000000
      max      158871.000000
Name: total, dtype: float64